In [ ]:
import netCDF4
import math
import pandas as pd

methanePath = '/home/joajohan/methaneAIS_data/ocean_ch4.nc'
aisPath = '/home/joajohan/methaneAIS_data/Global_2022_allPollutants_AmandaN.nc'

# Read methane data

In [ ]:
def readMethane(methanePath):
    methaneFile = netCDF4.Dataset(methanePath,'r')
    fdiff = [[x for x in v.data] for v in methaneFile.variables['Fch4_diffusive']]
    febul = [[x for x in v.data] for v in methaneFile.variables['Fch4_ebullitive']]

    longitudes = [v.data.flatten()[0] for v in methaneFile.variables['LON']]
    latitudes = [v.data.flatten()[0] for v in methaneFile.variables['LAT']]

    # Note that NaN values are filtered out
    return pd.DataFrame(
    [{
        'Latitude': lat,
        'Longitude': lng,
        'Fch4_diffusive': fdiff[lat_i][lng_i],
        'Fch4_ebullitive': febul[lat_i][lng_i]
    } for lat_i, lat in enumerate(latitudes) for lng_i, lng in enumerate(longitudes) if not math.isnan(fdiff[lat_i][lng_i]) and not math.isnan(febul[lat_i][lng_i])
    ])

In [ ]:
dfMethane = readMethane(methanePath)

# Read AIS data

In [ ]:
def readAIS(aisPath):
    aisFile = netCDF4.Dataset(aisPath,'r')
    travel = [v.data for v in aisFile.variables['TRAVEL']][0]

    longitudes = [v.data.flatten()[0] for v in aisFile.variables['longitude']]
    latitudes = [v.data.flatten()[0] for v in aisFile.variables['latitude']]

    # Note that zeros are filtered out
    return pd.DataFrame(
    [{
        'Latitude': lat,
        'Longitude': lng,
        'travel': travel[lat_i][lng_i],
    } for lat_i, lat in enumerate(latitudes) for lng_i, lng in enumerate(longitudes) if not travel[lat_i][lng_i] == 0
    ])

In [ ]:
dfAis = readAIS(aisPath)

# Plot data

In [ ]:
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [ ]:
def plotMap(df, value, cmap='RdYlBu_r', norm=colors.Normalize(), label=None, dpi=300):
    plt.figure(dpi=dpi)
    if label is None:
        label=value
    lat = df['Latitude'].values
    lon = df['Longitude'].values

    lon = np.array([v - 360 if v > 180 else v for v in lon])

    m = Basemap(projection = 'mill',
            llcrnrlat = -80,
            urcrnrlat = 80,
            llcrnrlon = -180,
            urcrnrlon = 180,
            resolution = 'c')

    m.drawparallels(np.arange(-90, 90, 30), labels=[True, False, False, False])
    m.drawmeridians(np.arange(-180, 180, 40), labels = [0,0,0,1])
    m.drawcoastlines(color='gray', linewidth=0.5)

    x, y = m(lon, lat)
    m.hexbin(x, y, 
        bins=None,
        norm = norm,
        C=df[value].values,
        gridsize=1000,
        alpha=1,
        edgecolors='none',
        cmap=cmap,
        reduce_C_function = np.sum
    )

    plt.colorbar(label=label, location="top")


## Diffusive

In [ ]:
plotMap(dfMethane, 'Fch4_diffusive', norm=colors.SymLogNorm(0.25, vmin=-0.5, vmax=100), label=r"Diffusive $CH_4$ $({mmol }m^{-2}y^{-1}$)")

## Ebulitive

In [ ]:
plotMap(dfMethane, 'Fch4_ebullitive', norm=colors.LogNorm(vmax=100), label=r"Ebulitive $CH_4$ (${mmol}$ $m^{-2}$ $y^{-1}$)")

In [ ]:
dfMethane['Fch4_ebullitive'].max()

## AIS travel data

In [ ]:
plotMap(dfAis, 'travel', cmap='YlOrRd', norm=colors.LogNorm(vmin=1, vmax=1000000), label=r"Kilometre travelled per grid cell")

# Try a merged plot

In [ ]:
diffusiveLimit = 5
travelLimit = 5

In [ ]:
from matplotlib.pyplot import figure

figure(dpi=300)

latMethane = dfMethane['Latitude'].values
lonMethane = dfMethane['Longitude'].values
lonMethane = np.array([v - 360 if v > 180 else v for v in lonMethane])

latAis = dfAis['Latitude'].values
lonAis = dfAis['Longitude'].values
lonAis = np.array([v - 360 if v > 180 else v for v in lonAis])

m = Basemap(projection = 'mill',
        llcrnrlat = -90,
        urcrnrlat = 90,
        llcrnrlon = -180,
        urcrnrlon = 180,
        resolution = 'c')

m.drawparallels(np.arange(-90, 90,30), labels=[True, False, False, False])

m.drawmeridians(np.arange(-180, 180, 40), labels = [0,0,0,1])
m.drawcoastlines(color='gray', linewidth=0.5)

x, y = m(np.concatenate([lonMethane, lonAis]), np.concatenate([latMethane, latAis]))
paddedTravelVals = np.concatenate([np.zeros_like(dfMethane['Fch4_diffusive'].values), dfAis['travel'].values])
paddedDiffVals = np.concatenate([dfMethane['Fch4_diffusive'].values, np.zeros_like(dfAis['travel'].values)])

cDiff = m.hexbin(x, y, bins='log', C=paddedDiffVals, gridsize=1000, alpha=1, edgecolors='none', cmap='RdYlBu_r')

cTravel = m.hexbin(x, y, bins='log', C=paddedTravelVals, gridsize=1000, alpha=1, edgecolors='none', cmap='RdYlBu_r')

aTravel = cTravel.get_array()
aDiff = cDiff.get_array()
aNew = np.zeros_like(aDiff)
for i in range(len(aNew)):
    if aTravel[i] >= travelLimit and aDiff[i] >= diffusiveLimit:
        aNew[i] = aTravel[i] * aDiff[i]

cTravel.set_array(aNew)

# Clear the other array to only show the merged one
cDiff.set_array(np.zeros_like(cDiff.get_array()))
